In [1]:
import os
import math
import shutil
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# Define the directories where the data is stored for training, validation, and testing
train_dir = '/kaggle/input/gold-coin-dataset2/train_data_directory/'
val_dir = '/kaggle/input/gold-coin-dataset2/validation_data_directory/'
test_dir = '/kaggle/input/gold-coin-dataset2/test_data_directory'

# Define the batch size and number of epochs for training
batch_size = 16
epochs = 10

# Define image dimensions
img_width, img_height = 224, 224

In [3]:
# Get the class names from the subdirectories of the training directory
class_names = sorted(os.listdir(train_dir))
class_names

['rsbl_100_995',
 'rsbl_100_999',
 'rsbl_10_995',
 'rsbl_10_999',
 'rsbl_1_995',
 'rsbl_1_999',
 'rsbl_20_995',
 'rsbl_20_999',
 'rsbl_2_995',
 'rsbl_2_999',
 'rsbl_50_995',
 'rsbl_50_999',
 'rsbl_5_995',
 'rsbl_5_999',
 'rsbl_8_995',
 'rsbl_8_999']

In [4]:
# Set up the data generator for training images with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
# Set up the data generator for validation and testing images (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names
)

Found 2585 images belonging to 16 classes.


In [7]:
# Load validation data from the directory with images
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  
)

Found 323 images belonging to 16 classes.


In [8]:
# Load testing data from the directory with images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names
)

Found 360 images belonging to 16 classes.


In [9]:
strategy = tf.distribute.MirroredStrategy()

In [17]:
from keras.models import *
from keras.layers import *
from keras.applications import VGG16

VGG = VGG16(input_shape = (224,224, 3), weights = 'imagenet', include_top = False)
   
VGG.trainable = False

model = Sequential()
model.add(VGG)
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(class_names), activation = 'sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10
81/81 [==============================] - 281s 3s/step - loss: 0.8283 - accuracy: 0.7729 - val_loss: 0.2901 - val_accuracy: 0.9288
Epoch 2/10
81/81 [==============================] - 194s 2s/step - loss: 0.2053 - accuracy: 0.9718 - val_loss: 0.1676 - val_accuracy: 0.9659
Epoch 3/10
81/81 [==============================] - 193s 2s/step - loss: 0.1105 - accuracy: 0.9849 - val_loss: 0.0728 - val_accuracy: 0.9938
Epoch 4/10
81/81 [==============================] - 202s 2s/step - loss: 0.0614 - accuracy: 0.9919 - val_loss: 0.1260 - val_accuracy: 0.9721
Epoch 5/10
81/81 [==============================] - 204s 3s/step - loss: 0.0447 - accuracy: 0.9942 - val_loss: 0.0319 - val_accuracy: 0.9938
Epoch 6/10
81/81 [==============================] - 206s 3s/step - loss: 0.0417 - accuracy: 0.9938 - val_loss: 0.0376 - val_accuracy: 0.9907
Epoch 7/10
81/81 [==============================] - 203s 3s/step - loss: 0.0391 - accuracy: 0.9934 - val_loss: 0.2622 - val_accuracy: 0.9443
Epoch 8/10
81

In [19]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
# Get class names and their corresponding indices from the data generator
class_names = list(test_generator.class_indices.keys())
class_indices = test_generator.class_indices

# Print class names and their corresponding indices
print("Class Names and Indices:")
for class_name, class_index in class_indices.items():
    print(f"{class_name}: {class_index}")

12/12 [==============================] - 35s 3s/step - loss: 0.0070 - accuracy: 1.0000
Test Loss: 0.006969274953007698
Test Accuracy: 1.0
Class Names and Indices:
rsbl_100_995: 0
rsbl_100_999: 1
rsbl_10_995: 2
rsbl_10_999: 3
rsbl_1_995: 4
rsbl_1_999: 5
rsbl_20_995: 6
rsbl_20_999: 7
rsbl_2_995: 8
rsbl_2_999: 9
rsbl_50_995: 10
rsbl_50_999: 11
rsbl_5_995: 12
rsbl_5_999: 13
rsbl_8_995: 14
rsbl_8_999: 15


In [20]:
model.save('gold_coin_classification_model_vgg16.h5')

In [22]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Define the class names (replace with your class names)
class_names = sorted(os.listdir("/kaggle/input/gold-coin-dataset2/train_data_directory/"))

# Load the trained model
model = load_model('gold_coin_classification_model_vgg16.h5')

# Define the test directory
test_dir = "/kaggle/input/gold-coin-dataset2/test_data_directory/"

# Get the list of class subdirectories in the test directory
class_dirs = sorted(os.listdir(test_dir))

# Test two random images from each class
for class_dir in class_dirs:
    # Create the full path to the class subdirectory
    class_path = os.path.join(test_dir, class_dir)

    # Get the list of image file names in the class subdirectory
    image_files = os.listdir(class_path)

    # Check if there are at least two images in the class
    if len(image_files) < 2:
        print(f"Not enough images in class '{class_dir}'. Skipping testing.")
        continue

    # Select two random images from the class
    random_image_files = random.sample(image_files, 2)

    for image_file in random_image_files:
        # Create the full path to the test image
        test_image_path = os.path.join(class_path, image_file)

        # Load and preprocess the test image
        img = load_img(test_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale pixel values to [0, 1]

        # Make the prediction
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_class_index]

        print("Test Image:", test_image_path)
        print("Predicted Class:", predicted_class)
        print()

1/1 [==============================] - 1s 794ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_995/IMG_20191114_142756.jpg
Predicted Class: rsbl_100_995

1/1 [==============================] - 0s 26ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_995/N8kDrQEMnD.jpg
Predicted Class: rsbl_100_995

1/1 [==============================] - 0s 22ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_999/VaOTOh6psu.jpg
Predicted Class: rsbl_100_999

1/1 [==============================] - 0s 23ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_999/IMG_20191114_143321.jpg
Predicted Class: rsbl_100_999

1/1 [==============================] - 0s 21ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_10_995/_F0A5535 - Copy (6).png
Predicted Class: rsbl_10_995

1/1 [==============================] - 0s 23ms/step
Test Image: /kaggle/input/gold-coin-datase

In [25]:
from keras import optimizers
from keras import optimizers

VGG.trainable = True
set_trainable = False

for layer in VGG.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10
81/81 [==============================] - 260s 3s/step - loss: 0.0531 - accuracy: 0.9907 - val_loss: 0.1602 - val_accuracy: 0.9443
Epoch 2/10
81/81 [==============================] - 211s 3s/step - loss: 0.0599 - accuracy: 0.9876 - val_loss: 0.0550 - val_accuracy: 0.9814
Epoch 3/10
81/81 [==============================] - 203s 3s/step - loss: 0.0405 - accuracy: 0.9954 - val_loss: 0.0242 - val_accuracy: 1.0000
Epoch 4/10
81/81 [==============================] - 198s 2s/step - loss: 0.0273 - accuracy: 0.9985 - val_loss: 0.0239 - val_accuracy: 0.9969
Epoch 5/10
81/81 [==============================] - 201s 2s/step - loss: 0.0241 - accuracy: 0.9957 - val_loss: 0.0357 - val_accuracy: 0.9845
Epoch 6/10
81/81 [==============================] - 206s 3s/step - loss: 0.0450 - accuracy: 0.9911 - val_loss: 0.0253 - val_accuracy: 0.9969
Epoch 7/10
81/81 [==============================] - 194s 2s/step - loss: 0.0410 - accuracy: 0.9907 - val_loss: 0.0267 - val_accuracy: 0.9969
Epoch 8/10
81

In [29]:
model.save('gold_coin_classification_model_vgg16_V2.h5')

In [30]:
# Define the class names (replace with your class names)
class_names = sorted(os.listdir("/kaggle/input/gold-coin-dataset2/train_data_directory/"))

# Load the trained model
model = load_model('gold_coin_classification_model_vgg16_V2.h5')

# Define the test directory
test_dir = "/kaggle/input/gold-coin-dataset2/test_data_directory/"

# Get the list of class subdirectories in the test directory
class_dirs = sorted(os.listdir(test_dir))

# Test two random images from each class
for class_dir in class_dirs:
    # Create the full path to the class subdirectory
    class_path = os.path.join(test_dir, class_dir)

    # Get the list of image file names in the class subdirectory
    image_files = os.listdir(class_path)

    # Check if there are at least two images in the class
    if len(image_files) < 2:
        print(f"Not enough images in class '{class_dir}'. Skipping testing.")
        continue

    # Select two random images from the class
    random_image_files = random.sample(image_files, 2)

    for image_file in random_image_files:
        # Create the full path to the test image
        test_image_path = os.path.join(class_path, image_file)

        # Load and preprocess the test image
        img = load_img(test_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale pixel values to [0, 1]

        # Make the prediction
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_class_index]

        print("Test Image:", test_image_path)
        print("Predicted Class:", predicted_class)
        print()

1/1 [==============================] - 0s 185ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_995/mZleE3miKa.jpg
Predicted Class: rsbl_100_995

1/1 [==============================] - 0s 22ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_995/IMG_20191119_150432.jpg
Predicted Class: rsbl_100_995

1/1 [==============================] - 0s 23ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_999/IMG_20191114_143321.jpg
Predicted Class: rsbl_100_999

1/1 [==============================] - 0s 21ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_999/IMG_20191114_143132.jpg
Predicted Class: rsbl_100_999

1/1 [==============================] - 0s 23ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_10_995/_F0A5535 - Copy (6).png
Predicted Class: rsbl_10_995

1/1 [==============================] - 0s 21ms/step
Test Image: /kaggle/input/gold-co